In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
temp_df = pd.read_csv('/content/IMDB Dataset.csv', on_bad_lines='skip', encoding='utf-8')

In [ ]:
df = temp_df.iloc[:10000]

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df['sentiment'].value_counts()

,count
sentiment,
positive,5008
negative,4992


In [ ]:
df.isnull().sum()

,0
review,0
sentiment,0


In [ ]:
df.duplicated().sum()

8

In [ ]:
df.drop_duplicates(inplace=True)

<ipython-input-14-16cdb8520be8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [ ]:
df.duplicated().sum()

0

In [ ]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [ ]:
df['review'] = df['review'].apply(remove_tags)

<ipython-input-17-00e93c2a3043>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [ ]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
9995,stuey unger was a card playing legend. he was ...,positive
9996,I thoroughly enjoyed Manna from Heaven. The ho...,positive
9997,I went to a screening of this movie and while ...,negative
9998,"I've read a few books about Bonnie and Clyde, ...",negative


In [ ]:
df['review'] = df['review'].apply(lambda x:x.lower())

<ipython-input-19-afca29351ee0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

sw_list = set(stopwords.words('english'))  # Use set for faster lookup

# Remove stopwords and join the words back into a string
df['review'] = df['review'].apply(lambda x: " ".join([word for word in x.split() if word not in sw_list]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<ipython-input-20-4cff895c7efb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: " ".join([word for word in x.split() if word not in sw_list]))


In [ ]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [ ]:
X

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,stuey unger card playing legend. quoted interv...
9996,thoroughly enjoyed manna heaven. hopes dreams ...
9997,went screening movie couple moments made laugh...
9998,"i've read books bonnie clyde, definitely accur..."


In [ ]:
y

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive
...,...
9995,positive
9996,positive
9997,negative
9998,negative


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [ ]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
X_train.shape

(7993, 1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

In [ ]:
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train_bow,y_train)

GaussianNB()

In [ ]:
y_pred = gnb.predict(X_test_bow)
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.6298149074537268

In [ ]:
confusion_matrix(y_test,y_pred)

array([[730, 293],
       [447, 529]])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8404202101050525

In [ ]:
cv = CountVectorizer(max_features=3000)
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()
rf = RandomForestClassifier()
rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8294147073536768

In [ ]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()
rf = RandomForestClassifier()
rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8479239619809905

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)
accuracy_score(y_test,y_pred)

0.848424212106053

In [ ]:
df

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive
...,...,...
9995,stuey unger card playing legend. quoted interv...,positive
9996,thoroughly enjoyed manna heaven. hopes dreams ...,positive
9997,went screening movie couple moments made laugh...,negative
9998,"i've read books bonnie clyde, definitely accur...",negative


In [ ]:
import gensim

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
story = []
for doc in df['review']:
  raw_sent = sent_tokenize(doc)
  for sent in raw_sent:
    story.append(simple_preprocess(sent))

In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
model.build_vocab(story)

In [ ]:
model.train(story,total_examples=model.corpus_count,epochs=model.epochs)

(5967291, 6307680)

In [ ]:
len(model.wv.index_to_key)

32192

In [ ]:
def document_vector(doc):
  doc = [word for word in doc.split() if word in model.wv.index_to_key]
  return np.mean(model.wv[doc],axis=0)

In [ ]:
document_vector(df['review'].values[0])

array([-0.14593993,  0.40862948,  0.17675453,  0.16930145,  0.01302413,
       -0.7074475 ,  0.13770665,  0.89748603, -0.2917154 , -0.1752167 ,
       -0.16045676, -0.62196547,  0.05897034,  0.23741077,  0.1425173 ,
       -0.2023814 ,  0.14831012, -0.5388957 , -0.16787477, -0.46890083,
        0.09244834,  0.2713916 ,  0.30732384, -0.15911676,  0.07612631,
        0.04502257, -0.33048666, -0.12051699, -0.32783532, -0.00172502,
        0.32046348, -0.01608482, -0.14850087, -0.28959873, -0.10123888,
        0.49707025,  0.18190272, -0.43787736, -0.3775028 , -0.75568473,
        0.01503383, -0.2701056 , -0.08922847, -0.1698589 ,  0.45285612,
       -0.16032463, -0.23565389, -0.02876217,  0.16345799,  0.15656367,
        0.26770508, -0.4868256 , -0.03240695,  0.03886071, -0.1512481 ,
        0.06236497,  0.19617587,  0.05384849, -0.37197244, -0.00163168,
        0.07988233,  0.19375944, -0.15185173,  0.03737757, -0.3875413 ,
        0.34078348,  0.07233703,  0.3839368 , -0.46182615,  0.53

In [ ]:
from tqdm import tqdm

In [ ]:
X = []
for doc in tqdm(df['review'].values):
  X.append(document_vector(doc))

100%|██████████| 9992/9992 [09:31<00:00, 17.49it/s]


In [ ]:
X = np.array(X)

In [ ]:
X.shape

(9992, 100)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(df['sentiment'])

In [ ]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7643821910955477